In [ ]:
import matplotlib.pyplot as plt
import torch as t
from torchtyping import TensorType, patch_typeguard
patch_typeguard()
from typeguard import typechecked
from typing import List, Tuple

In [ ]:
DEVICE = "cuda" if t.cuda.is_available() else "cpu"
DEVICE

In [ ]:
@typechecked
def strict_upper_triangular_mask(seq_len: int, device=DEVICE) -> TensorType["seq_len", "seq_len"]:
    return t.arange(seq_len, device=device).unsqueeze(1) < t.arange(seq_len, device=device).unsqueeze(0)

strict_upper_triangular_mask(3)

In [ ]:
@typechecked
def probs(q: TensorType["seq_len", "head_size"], k: TensorType["seq_len", "head_size"]) -> TensorType["seq_len", "seq_len"]:
    seq_len, head_size = q.shape
    scores = q @ k.T / t.sqrt(t.tensor(head_size, device=q.device))
    masked_scores = t.where(strict_upper_triangular_mask(seq_len, device=scores.device), t.tensor(-1e4, device=scores.device), scores)
    return t.softmax(masked_scores, dim=-1)

In [ ]:
@typechecked
def plot_probs_with_1d_scores(qks: List[Tuple[List[float], List[float]]]):
    probses = [
        m
        for q, k in qks
        for m in [
            probs(q=t.tensor([[x] for x in q]), k=t.tensor([[x] for x in k])),
            t.ones(len(q), 1) / 2,
        ]
    ]
    plt.imshow(t.cat(probses, dim=1).detach().numpy())
    plt.show()

In [ ]:
plot_probs_with_1d_scores([
    ([1., 1., 1.], [1., 10., 100.]),
    ([1., 1., 1.], [1., 10., 0.]),
    ([1., 1., -1.], [1., 10., 100.]),
    ([1., 1., 1.], [10., 0., 100.]),
    ([1., 1., -1.], [10., 0., 10.]),
    ([1., 1., 1.], [10., 0., 0.]),
])

In [ ]:
plot_probs_with_1d_scores([
    ([1., 1., 1., 1.], [1., 10., 100., 1000.]),
    ([1., 1., 1., 1.], [1., 10., 100., 0.]),
    ([1., -1., 1., -1.], [10., 1., 100., 100.]),
    ([1., -1., -1., 1.], [100., 10., 1., 0.]),
    ([1., 1., 1., 1.], [1., 10., 1., 100.]),
    ([1., 1., 1., -1.], [10., 100., 1., 10.]),
    ([1., 1., 1., 1.], [10., 100., 1., 1.]),
    ([1., 1., 1., -1.], [1., 100., 10., 10.]),
])
plot_probs_with_1d_scores([
    ([1., 1., -1., 1.], [1., 10., 100., 1000.]),
    ([1., 1., -1., 1.], [1., 10., 100., 1.]),
    ([1., 1., -1., 1.], [10., 1000., 100., 1.]),
    ([1., -1., 1., 1.], [10., 1., 1., 1.]),
    ([1., -1., 1., 1.], [1., 10., 100., 1000.]),
    ([1., 1., 1., 1.], [10., 1., 100., 1.]),
    ([1., 1., 1., -1.], [10., 1., 100., 10.]),
    ([1., 1., -1., 1.], [100., 10., 1., 10.]),
])
plot_probs_with_1d_scores([
    ([1., 1., -1., 1.], [10., 1., 10., 100.]),
    ([1., 1., -1., 1.], [10., 1., 100., 1.]),
    ([1., 1., -1., -1.], [10., 1., 10., 10.]),
    ([1., 1., -1., 1.], [100., 1., 10., 10.]),
    ([1., 1., 1., 1.], [10., 1., 1., 100.]),
    ([1., 1., 1., -1.], [100., 10., 1., 10.]),
    ([1., 1., 1., -1.], [100., 1., 10., 10.]),
    ([1., 1., 1., 1.], [10., 1., 1., 1.]),
])